In [1]:
import pandas as pd
from google.cloud import bigquery
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from google.cloud import storage
from tensorflow.keras import callbacks
from tensorflow.keras import Sequential, Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

2025-06-05 10:58:29.747703: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 10:58:29.773755: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 10:58:30.013111: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 10:58:30.168800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749117510.316485    4764 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749117510.35

In [2]:
# Variables
BUCKET_NAME="skin_scan_mohnatz"
GCP_PROJECT = "skin-scan-461716"
BQ_DATASET = "skin_scan"

In [3]:
def get_data_from_bq(
        gcp_project:str,
        query:str,
        data_has_header=True
    ) -> pd.DataFrame:
    """
    Retrieve `query` data from BigQuery
    """
    client = bigquery.Client(project=gcp_project)
    query_job = client.query(query)
    result = query_job.result()
    df = result.to_dataframe()
    return df

query_metadata = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.train_metadata
    """

In [14]:
metadata = get_data_from_bq(f"{GCP_PROJECT}", query_metadata)
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,index
0,HAM_0006706,ISIC_0031725,nv,histo,NaN,female,abdomen,8410
1,HAM_0002610,ISIC_0026876,vasc,consensus,0.0,female,abdomen,2388
2,HAM_0002610,ISIC_0030606,vasc,consensus,0.0,female,abdomen,2389
3,HAM_0003872,ISIC_0028903,nv,consensus,0.0,female,abdomen,9329
4,HAM_0002034,ISIC_0027954,bkl,consensus,5.0,male,abdomen,1035


In [15]:
import sys
import os

# Add parent directory to Python path
sys.path.append(os.path.abspath(".."))

# Now you can import the module
import preprocessing

In [18]:
metadata.drop(columns=[col for col in ['dx_type','lesion_id'] if col in metadata.columns])

,image_id,dx,age,sex,localization,index
0,ISIC_0031725,nv,NaN,female,abdomen,8410
1,ISIC_0026876,vasc,0.0,female,abdomen,2388
2,ISIC_0030606,vasc,0.0,female,abdomen,2389
3,ISIC_0028903,nv,0.0,female,abdomen,9329
4,ISIC_0027954,bkl,5.0,male,abdomen,1035
...,...,...,...,...,...,...
10010,ISIC_0029610,akiec,85.0,male,upper extremity,9840
10011,ISIC_0032397,akiec,85.0,male,upper extremity,9865
10012,ISIC_0029268,akiec,85.0,male,upper extremity,9875
10013,ISIC_0028763,akiec,85.0,male,upper extremity,9950


In [ ]:
print("Make a small change"